# Ads Platform Data Segment Validation with 2% Threshold

This notebook performs **segment-level validation** for ads platform data with a **2% tolerance threshold** and **placement name normalization**.

**Files:**
- Growth: `growth/merged_ads_platform(growth).xlsx` (1,264 rows)
- Gold: `gold/merged_ads_platform(gold).xlsx` (10,263 rows)

**Key Feature:** Placement name normalization (20 mappings)
- Growth uses human-readable names: "Facebook Reels", "Instagram Stories"
- Gold uses system names: "facebook_reels", "instagram_stories"

**Validation Segments:**
- Overall Totals
- By Date
- By Campaign
- By Platform
- By Placement (with normalization)
- By Campaign + Date

## Configuration: Set Threshold

In [1]:
# CONFIGURATION: Set your threshold here
THRESHOLD_PERCENT = 2.0  # Accept differences up to 2%

print("="*80)
print("ADS PLATFORM DATA VALIDATION CONFIGURATION")
print("="*80)
print(f"\nThreshold: {THRESHOLD_PERCENT}%")
print(f"Differences under {THRESHOLD_PERCENT}% will be marked as MATCHED")
print("\nYou can change THRESHOLD_PERCENT above to adjust tolerance")

ADS PLATFORM DATA VALIDATION CONFIGURATION

Threshold: 2.0%
Differences under 2.0% will be marked as MATCHED

You can change THRESHOLD_PERCENT above to adjust tolerance


## Step 1: Import Libraries

In [2]:
# Install openpyxl if needed
import sys
!{sys.executable} -m pip install openpyxl -q

import pandas as pd
import numpy as np
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Libraries imported successfully")
print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✓ Libraries imported successfully
Analysis started: 2025-12-20 23:10:23


## Step 2: Define Placement Mapping

In [3]:
# Placement name mapping: Growth → Gold
PLACEMENT_MAPPING = {
    # Facebook placements
    'Ads on Facebook Reels': 'facebook_reels_overlay',
    'Facebook Reels': 'facebook_reels',
    'Facebook Stories': 'facebook_stories',
    'Facebook notifications': 'facebook_notification',
    'Facebook profile feed': 'facebook_profile_feed',
    'In-stream reels': 'instream_video',
    'Marketplace': 'marketplace',
    'Right column': 'right_hand_column',
    'Search results': 'search',
    
    # Instagram placements
    'Explore': 'instagram_explore',
    'Explore home': 'instagram_explore_grid_home',
    'Instagram Reels': 'instagram_reels',
    'Instagram Stories': 'instagram_stories',
    'Instagram search results': 'instagram_search',
    
    # Messenger placements
    'Messenger Stories': 'messenger_stories',
    'Messenger inbox': 'messenger_inbox',
    
    # Audience Network placements
    'Native, banner and interstitial': 'an_classic',
    'Rewarded video': 'rewarded_video',
    
    # Threads placements
    'Threads feed': 'threads_feed',
    
    # Feed (common across platforms)
    'Feed': 'feed'
}

print(f"✓ Placement mapping loaded: {len(PLACEMENT_MAPPING)} mappings")
print("\nMappings:")
for growth, gold in sorted(PLACEMENT_MAPPING.items())[:5]:
    print(f"  {growth:40} → {gold}")
print(f"  ... and {len(PLACEMENT_MAPPING)-5} more")

✓ Placement mapping loaded: 20 mappings

Mappings:
  Ads on Facebook Reels                    → facebook_reels_overlay
  Explore                                  → instagram_explore
  Explore home                             → instagram_explore_grid_home
  Facebook Reels                           → facebook_reels
  Facebook Stories                         → facebook_stories
  ... and 15 more


## Step 3: Load and Prepare Data

In [4]:
# Load Growth Excel
print("Loading Growth Excel...")
growth_df = pd.read_excel("growth/merged_ads_platform(growth).xlsx")

# Convert Day to datetime format
growth_df['Day'] = pd.to_datetime(growth_df['Day']).dt.strftime('%Y-%m-%d')

# NORMALIZE PLACEMENT NAMES
growth_df['Placement_normalized'] = growth_df['Placement'].map(PLACEMENT_MAPPING)
unmapped = growth_df[growth_df['Placement_normalized'].isna()]['Placement'].unique()
if len(unmapped) > 0:
    print(f"⚠ Warning: {len(unmapped)} unmapped placements found:")
    for p in unmapped:
        print(f"  • {p}")
    growth_df['Placement_normalized'] = growth_df['Placement_normalized'].fillna(growth_df['Placement'])

print(f"✓ Growth loaded: {len(growth_df):,} rows")
print(f"  Columns: {growth_df.columns.tolist()}")

# Load Gold Excel
print("\nLoading Gold Excel...")
gold_df = pd.read_excel("gold/merged_ads_platform(gold).xlsx")

# Convert Day to string format for consistent comparison
gold_df['Day'] = pd.to_datetime(gold_df['Day']).dt.strftime('%Y-%m-%d')
gold_df['Placement_normalized'] = gold_df['Placement']  # Already in normalized format

print(f"✓ Gold loaded: {len(gold_df):,} rows")
print(f"  Columns: {gold_df.columns.tolist()}")

print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)
print(f"\nGrowth Date Range: {growth_df['Day'].min()} to {growth_df['Day'].max()}")
print(f"Gold Date Range: {gold_df['Day'].min()} to {gold_df['Day'].max()}")
print(f"\nGrowth Unique Campaigns: {growth_df['Campaign name'].nunique()}")
print(f"Gold Unique Campaigns: {gold_df['Campaign name'].nunique()}")
print(f"\nGrowth Unique Platforms: {sorted(growth_df['Platform'].unique())}")
print(f"Gold Unique Platforms: {sorted(gold_df['Platform'].unique())}")
print(f"\nGrowth Unique Placements (original): {growth_df['Placement'].nunique()}")
print(f"Growth Unique Placements (normalized): {growth_df['Placement_normalized'].nunique()}")
print(f"Gold Unique Placements: {gold_df['Placement_normalized'].nunique()}")

Loading Growth Excel...
✓ Growth loaded: 1,264 rows
  Columns: ['Day', 'Campaign name', 'Platform', 'Placement', 'Amount spent (INR)', 'Impressions', 'Link clicks', 'Purchases conversion value', 'Purchases', 'Reporting starts', 'Reporting ends', 'Placement_normalized']

Loading Gold Excel...
✓ Gold loaded: 10,263 rows
  Columns: ['Day', 'Campaign name', 'Platform', 'Placement', 'Amount spent (INR)', 'Impressions', 'Link clicks', 'Purchases conversion value', 'Purchases', 'Reporting starts', 'Reporting ends', 'Placement_normalized']

DATA SUMMARY

Growth Date Range: 2025-11-01 to 2025-11-30
Gold Date Range: 2025-11-01 to 2025-11-30

Growth Unique Campaigns: 4
Gold Unique Campaigns: 4

Growth Unique Platforms: ['audience_network', 'facebook', 'instagram', 'messenger', 'threads']
Gold Unique Platforms: ['audience_network', 'facebook', 'instagram', 'messenger', 'threads', 'unknown']

Growth Unique Placements (original): 20
Growth Unique Placements (normalized): 20
Gold Unique Placements: 2

## Step 4: Overall Totals Comparison

In [5]:
print("="*80)
print(f"OVERALL TOTALS COMPARISON (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Calculate totals
growth_totals = growth_df[['Amount spent (INR)', 'Impressions', 'Link clicks', 'Purchases', 'Purchases conversion value']].sum()
gold_totals = gold_df[['Amount spent (INR)', 'Impressions', 'Link clicks', 'Purchases', 'Purchases conversion value']].sum()

# Create comparison dataframe
overall_comparison = pd.DataFrame({
    'Metric': ['Amount Spent (INR)', 'Impressions', 'Link Clicks', 'Purchases', 'Conversion Value'],
    'Growth': [
        growth_totals['Amount spent (INR)'],
        growth_totals['Impressions'],
        growth_totals['Link clicks'],
        growth_totals['Purchases'],
        growth_totals['Purchases conversion value']
    ],
    'Gold': [
        gold_totals['Amount spent (INR)'],
        gold_totals['Impressions'],
        gold_totals['Link clicks'],
        gold_totals['Purchases'],
        gold_totals['Purchases conversion value']
    ],
})

overall_comparison['Difference'] = overall_comparison['Growth'] - overall_comparison['Gold']
overall_comparison['Diff %'] = (overall_comparison['Difference'] / overall_comparison['Gold'] * 100).round(2)
overall_comparison['Match'] = overall_comparison['Diff %'].abs() <= THRESHOLD_PERCENT
overall_comparison['Status'] = overall_comparison['Match'].apply(lambda x: '✓ PASS' if x else '✗ FAIL')

display(overall_comparison)

# Summary
matches = overall_comparison['Match'].sum()
print(f"\n✓ Matches (within {THRESHOLD_PERCENT}%): {matches}/5 metrics")
if matches == 5:
    print(f"✓✓✓ ALL OVERALL TOTALS MATCH (within {THRESHOLD_PERCENT}% threshold)! ✓✓✓")
else:
    print(f"⚠ {5-matches} metric(s) exceed {THRESHOLD_PERCENT}% threshold")

OVERALL TOTALS COMPARISON (with 2.0% threshold)


,Metric,Growth,Gold,Difference,Diff %,Match,Status
0,Amount Spent (INR),566296.98,566308.97,-11.99,-0.00,True,✓ PASS
1,Impressions,5076744.00,5076850.00,-106.00,-0.00,True,✓ PASS
2,Link Clicks,82102.00,82104.00,-2.00,-0.00,True,✓ PASS
3,Purchases,1070.00,0.00,1070.00,inf,False,✗ FAIL
4,Conversion Value,1841179.52,0.00,1841179.52,inf,False,✗ FAIL



✓ Matches (within 2.0%): 3/5 metrics
⚠ 2 metric(s) exceed 2.0% threshold


## Step 5: Validation by Date

In [6]:
print("="*80)
print(f"SEGMENT VALIDATION: BY DATE (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Aggregate by date
growth_by_date = growth_df.groupby('Day').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
growth_by_date.columns = ['Day', 'amount_growth', 'impressions_growth', 'clicks_growth']

gold_by_date = gold_df.groupby('Day').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
gold_by_date.columns = ['Day', 'amount_gold', 'impressions_gold', 'clicks_gold']

# Merge and compare
date_comparison = pd.merge(growth_by_date, gold_by_date, on='Day', how='inner')

# Calculate percentage differences
date_comparison['amount_diff_pct'] = ((date_comparison['amount_growth'] - date_comparison['amount_gold']) / date_comparison['amount_gold'] * 100).round(2)
date_comparison['impr_diff_pct'] = ((date_comparison['impressions_growth'] - date_comparison['impressions_gold']) / date_comparison['impressions_gold'] * 100).round(2)
date_comparison['clicks_diff_pct'] = ((date_comparison['clicks_growth'] - date_comparison['clicks_gold']) / date_comparison['clicks_gold'] * 100).round(2)

# Apply threshold matching
date_comparison['perfect_match'] = (
    (date_comparison['amount_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (date_comparison['impr_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (date_comparison['clicks_diff_pct'].abs() <= THRESHOLD_PERCENT)
)
date_comparison['status'] = date_comparison['perfect_match'].apply(lambda x: '✓ PASS' if x else '✗ FAIL')

print(f"\nTotal dates compared: {len(date_comparison)}")
print(f"✓ Matches (within {THRESHOLD_PERCENT}%): {date_comparison['perfect_match'].sum()}")
print(f"✗ Exceeds threshold: {(~date_comparison['perfect_match']).sum()}")

print("\nDetailed comparison:")
display(date_comparison[['Day', 'amount_growth', 'amount_gold', 'amount_diff_pct',
                          'impressions_growth', 'impressions_gold', 'impr_diff_pct',
                          'clicks_growth', 'clicks_gold', 'clicks_diff_pct', 'status']].sort_values('Day'))

SEGMENT VALIDATION: BY DATE (with 2.0% threshold)

Total dates compared: 30
✓ Matches (within 2.0%): 30
✗ Exceeds threshold: 0

Detailed comparison:


,Day,amount_growth,amount_gold,amount_diff_pct,impressions_growth,impressions_gold,impr_diff_pct,clicks_growth,clicks_gold,clicks_diff_pct,status
0,2025-11-01,13084.64,13084.64,-0.00,345244,345244.00,0.00,1655.00,1655.00,0.00,✓ PASS
1,2025-11-02,14197.48,14197.48,0.00,73582,73582.00,0.00,483.00,483.00,0.00,✓ PASS
2,2025-11-03,11293.66,11293.66,0.00,61257,61257.00,0.00,608.00,608.00,0.00,✓ PASS
3,2025-11-04,26489.13,26489.13,0.00,275517,275517.00,0.00,3139.00,3139.00,0.00,✓ PASS
4,2025-11-05,17204.24,17204.24,-0.00,120019,120019.00,0.00,2056.00,2056.00,0.00,✓ PASS
5,2025-11-06,16253.66,16253.66,0.00,111832,111832.00,0.00,1795.00,1795.00,0.00,✓ PASS
6,2025-11-07,24867.92,24867.92,0.00,232865,232865.00,0.00,2887.00,2887.00,0.00,✓ PASS
7,2025-11-08,22173.71,22173.71,0.00,150184,150184.00,0.00,1938.00,1938.00,0.00,✓ PASS
8,2025-11-09,21629.76,21629.76,0.00,180267,180267.00,0.00,1723.00,1723.00,0.00,✓ PASS
9,2025-11-10,16802.72,16802.72,0.00,119316,119316.00,0.00,1846.00,1846.00,0.00,✓ PASS


## Step 6: Validation by Campaign

In [7]:
print("="*80)
print(f"SEGMENT VALIDATION: BY CAMPAIGN (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Aggregate by campaign
growth_by_campaign = growth_df.groupby('Campaign name').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
growth_by_campaign.columns = ['Campaign name', 'amount_growth', 'impressions_growth', 'clicks_growth']

gold_by_campaign = gold_df.groupby('Campaign name').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
gold_by_campaign.columns = ['Campaign name', 'amount_gold', 'impressions_gold', 'clicks_gold']

# Merge and compare
campaign_comparison = pd.merge(growth_by_campaign, gold_by_campaign, on='Campaign name', how='inner')

# Calculate percentage differences
campaign_comparison['amount_diff_pct'] = ((campaign_comparison['amount_growth'] - campaign_comparison['amount_gold']) / campaign_comparison['amount_gold'] * 100).round(2)
campaign_comparison['impr_diff_pct'] = ((campaign_comparison['impressions_growth'] - campaign_comparison['impressions_gold']) / campaign_comparison['impressions_gold'] * 100).round(2)
campaign_comparison['clicks_diff_pct'] = ((campaign_comparison['clicks_growth'] - campaign_comparison['clicks_gold']) / campaign_comparison['clicks_gold'] * 100).round(2)

# Apply threshold matching
campaign_comparison['perfect_match'] = (
    (campaign_comparison['amount_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (campaign_comparison['impr_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (campaign_comparison['clicks_diff_pct'].abs() <= THRESHOLD_PERCENT)
)
campaign_comparison['status'] = campaign_comparison['perfect_match'].apply(lambda x: '✓ PASS' if x else '✗ FAIL')

print(f"\nTotal campaigns compared: {len(campaign_comparison)}")
print(f"✓ Matches (within {THRESHOLD_PERCENT}%): {campaign_comparison['perfect_match'].sum()}")
print(f"✗ Exceeds threshold: {(~campaign_comparison['perfect_match']).sum()}")

print("\nDetailed comparison:")
display(campaign_comparison[['Campaign name', 'amount_growth', 'amount_gold', 'amount_diff_pct',
                              'impressions_growth', 'impressions_gold', 'impr_diff_pct',
                              'clicks_growth', 'clicks_gold', 'clicks_diff_pct', 'status']].sort_values('Campaign name'))

SEGMENT VALIDATION: BY CAMPAIGN (with 2.0% threshold)

Total campaigns compared: 4
✓ Matches (within 2.0%): 4
✗ Exceeds threshold: 0

Detailed comparison:


,Campaign name,amount_growth,amount_gold,amount_diff_pct,impressions_growth,impressions_gold,impr_diff_pct,clicks_growth,clicks_gold,clicks_diff_pct,status
0,Ikonic -Scalp-Massager-Amazon-1-Nov2025,8967.01,8967.01,0.00,56943,56943.00,0.00,254.00,254.00,0.00,✓ PASS
1,Ikonic ME | Sales Retargeting,207786.87,207787.02,-0.00,1514495,1514502.00,-0.00,21140.00,21140.00,0.00,✓ PASS
2,Ikonic ME | Sales Prospecting,193185.80,193189.74,-0.00,1752762,1752793.00,-0.00,26509.00,26511.00,-0.01,✓ PASS
3,Ikonic Me | Sales Catalogue,156357.30,156365.20,-0.01,1752544,1752612.00,-0.00,34199.00,34199.00,0.00,✓ PASS


## Step 7: Validation by Platform

In [8]:
print("="*80)
print(f"SEGMENT VALIDATION: BY PLATFORM (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Aggregate by platform
growth_by_platform = growth_df.groupby('Platform').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
growth_by_platform.columns = ['Platform', 'amount_growth', 'impressions_growth', 'clicks_growth']

gold_by_platform = gold_df.groupby('Platform').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
gold_by_platform.columns = ['Platform', 'amount_gold', 'impressions_gold', 'clicks_gold']

# Merge and compare
platform_comparison = pd.merge(growth_by_platform, gold_by_platform, on='Platform', how='inner')

# Calculate percentage differences
platform_comparison['amount_diff_pct'] = ((platform_comparison['amount_growth'] - platform_comparison['amount_gold']) / platform_comparison['amount_gold'] * 100).round(2)
platform_comparison['impr_diff_pct'] = ((platform_comparison['impressions_growth'] - platform_comparison['impressions_gold']) / platform_comparison['impressions_gold'] * 100).round(2)
platform_comparison['clicks_diff_pct'] = ((platform_comparison['clicks_growth'] - platform_comparison['clicks_gold']) / platform_comparison['clicks_gold'] * 100).round(2)

# Apply threshold matching
platform_comparison['perfect_match'] = (
    (platform_comparison['amount_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (platform_comparison['impr_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (platform_comparison['clicks_diff_pct'].abs() <= THRESHOLD_PERCENT)
)
platform_comparison['status'] = platform_comparison['perfect_match'].apply(lambda x: '✓ PASS' if x else '✗ FAIL')

print(f"\nTotal platforms compared: {len(platform_comparison)}")
print(f"✓ Matches (within {THRESHOLD_PERCENT}%): {platform_comparison['perfect_match'].sum()}")
print(f"✗ Exceeds threshold: {(~platform_comparison['perfect_match']).sum()}")

print("\nDetailed comparison:")
display(platform_comparison.sort_values('Platform'))

SEGMENT VALIDATION: BY PLATFORM (with 2.0% threshold)

Total platforms compared: 5
✓ Matches (within 2.0%): 5
✗ Exceeds threshold: 0

Detailed comparison:


,Platform,amount_growth,impressions_growth,clicks_growth,amount_gold,impressions_gold,clicks_gold,amount_diff_pct,impr_diff_pct,clicks_diff_pct,perfect_match,status
0,audience_network,237.96,652,96.00,237.96,652.00,96.00,0.00,0.00,0.00,True,✓ PASS
1,facebook,156903.77,1287432,26498.00,156908.82,1287535.00,26500.00,-0.00,-0.01,-0.01,True,✓ PASS
2,instagram,409134.03,3785298,55506.00,409140.98,3785301.00,55506.00,-0.00,-0.00,0.00,True,✓ PASS
3,messenger,15.30,3094,1.00,15.30,3094.00,1.00,0.00,0.00,0.00,True,✓ PASS
4,threads,5.91,268,1.00,5.91,268.00,1.00,0.00,0.00,0.00,True,✓ PASS


## Step 8: Validation by Placement (with Normalization)

In [9]:
print("="*80)
print(f"SEGMENT VALIDATION: BY PLACEMENT (NORMALIZED) (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Aggregate by normalized placement
growth_by_placement = growth_df.groupby('Placement_normalized').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
growth_by_placement.columns = ['Placement', 'amount_growth', 'impressions_growth', 'clicks_growth']

gold_by_placement = gold_df.groupby('Placement_normalized').agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
gold_by_placement.columns = ['Placement', 'amount_gold', 'impressions_gold', 'clicks_gold']

# Merge and compare
placement_comparison = pd.merge(growth_by_placement, gold_by_placement, on='Placement', how='inner')

# Calculate percentage differences
placement_comparison['amount_diff_pct'] = ((placement_comparison['amount_growth'] - placement_comparison['amount_gold']) / placement_comparison['amount_gold'] * 100).round(2)
placement_comparison['impr_diff_pct'] = ((placement_comparison['impressions_growth'] - placement_comparison['impressions_gold']) / placement_comparison['impressions_gold'] * 100).round(2)
placement_comparison['clicks_diff_pct'] = ((placement_comparison['clicks_growth'] - placement_comparison['clicks_gold']) / placement_comparison['clicks_gold'] * 100).round(2)

# Apply threshold matching
placement_comparison['perfect_match'] = (
    (placement_comparison['amount_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (placement_comparison['impr_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (placement_comparison['clicks_diff_pct'].abs() <= THRESHOLD_PERCENT)
)
placement_comparison['status'] = placement_comparison['perfect_match'].apply(lambda x: '✓ PASS' if x else '✗ FAIL')

print(f"\nTotal placements compared: {len(placement_comparison)}")
print(f"✓ Matches (within {THRESHOLD_PERCENT}%): {placement_comparison['perfect_match'].sum()}")
print(f"✗ Exceeds threshold: {(~placement_comparison['perfect_match']).sum()}")

print("\nTop 20 placements by amount spent:")
display(placement_comparison.nlargest(20, 'amount_growth')[['Placement', 'amount_growth', 'amount_gold', 'amount_diff_pct',
                                                              'impressions_growth', 'impressions_gold', 'impr_diff_pct',
                                                              'clicks_growth', 'clicks_gold', 'clicks_diff_pct', 'status']])

SEGMENT VALIDATION: BY PLACEMENT (NORMALIZED) (with 2.0% threshold)

Total placements compared: 20
✓ Matches (within 2.0%): 17
✗ Exceeds threshold: 3

Top 20 placements by amount spent:


,Placement,amount_growth,amount_gold,amount_diff_pct,impressions_growth,impressions_gold,impr_diff_pct,clicks_growth,clicks_gold,clicks_diff_pct,status
6,feed,275731.62,275743.25,-0.00,2031933,2032000.00,-0.00,39526.00,39526.00,0.00,✓ PASS
9,instagram_reels,165698.61,165699.35,-0.00,2024777,2024782.00,-0.00,24161.00,24161.00,0.00,✓ PASS
11,instagram_stories,69475.83,69476.48,-0.00,446347,446345.00,0.00,10040.00,10040.00,0.00,✓ PASS
3,facebook_reels,42766.58,42766.04,0.00,430758,430792.00,-0.01,6406.00,6408.00,-0.03,✓ PASS
5,facebook_stories,8606.70,8606.15,0.01,45239,45240.00,-0.00,1096.00,1096.00,0.00,✓ PASS
12,instream_video,1266.04,1266.03,0.00,9469,9469.00,0.00,155.00,155.00,0.00,✓ PASS
7,instagram_explore,816.21,816.24,-0.00,16276,16276.00,0.00,203.00,203.00,0.00,✓ PASS
13,marketplace,651.58,651.59,-0.00,22921,22922.00,-0.00,232.00,232.00,0.00,✓ PASS
8,instagram_explore_grid_home,554.49,554.51,-0.00,33763,33763.00,0.00,106.00,106.00,0.00,✓ PASS
2,facebook_profile_feed,307.89,307.89,-0.00,6784,6784.00,0.00,61.00,61.00,0.00,✓ PASS


## Step 9: Validation by Campaign + Date

In [10]:
print("="*80)
print(f"SEGMENT VALIDATION: BY CAMPAIGN + DATE (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)

# Aggregate by campaign and date
growth_by_camp_date = growth_df.groupby(['Campaign name', 'Day']).agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
growth_by_camp_date.columns = ['Campaign name', 'Day', 'amount_growth', 'impressions_growth', 'clicks_growth']

gold_by_camp_date = gold_df.groupby(['Campaign name', 'Day']).agg({
    'Amount spent (INR)': 'sum',
    'Impressions': 'sum',
    'Link clicks': 'sum'
}).reset_index()
gold_by_camp_date.columns = ['Campaign name', 'Day', 'amount_gold', 'impressions_gold', 'clicks_gold']

# Merge and compare
camp_date_comparison = pd.merge(growth_by_camp_date, gold_by_camp_date, on=['Campaign name', 'Day'], how='inner')

# Calculate percentage differences
camp_date_comparison['amount_diff_pct'] = ((camp_date_comparison['amount_growth'] - camp_date_comparison['amount_gold']) / camp_date_comparison['amount_gold'] * 100).round(2)
camp_date_comparison['impr_diff_pct'] = ((camp_date_comparison['impressions_growth'] - camp_date_comparison['impressions_gold']) / camp_date_comparison['impressions_gold'] * 100).round(2)
camp_date_comparison['clicks_diff_pct'] = ((camp_date_comparison['clicks_growth'] - camp_date_comparison['clicks_gold']) / camp_date_comparison['clicks_gold'] * 100).round(2)

# Apply threshold matching
camp_date_comparison['perfect_match'] = (
    (camp_date_comparison['amount_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (camp_date_comparison['impr_diff_pct'].abs() <= THRESHOLD_PERCENT) & 
    (camp_date_comparison['clicks_diff_pct'].abs() <= THRESHOLD_PERCENT)
)

print(f"\nTotal campaign+date segments: {len(camp_date_comparison)}")
print(f"✓ Matches (within {THRESHOLD_PERCENT}%): {camp_date_comparison['perfect_match'].sum()}")
print(f"✗ Exceeds threshold: {(~camp_date_comparison['perfect_match']).sum()}")

# Show mismatches if any
if (~camp_date_comparison['perfect_match']).sum() > 0:
    print("\nSample mismatches (first 10):")
    mismatches = camp_date_comparison[~camp_date_comparison['perfect_match']]
    display(mismatches[['Campaign name', 'Day', 'amount_diff_pct', 'impr_diff_pct', 'clicks_diff_pct']].head(10))
else:
    print("\n✓✓✓ ALL CAMPAIGN+DATE SEGMENTS MATCH! ✓✓✓")

SEGMENT VALIDATION: BY CAMPAIGN + DATE (with 2.0% threshold)

Total campaign+date segments: 101
✓ Matches (within 2.0%): 100
✗ Exceeds threshold: 1

Sample mismatches (first 10):


,Campaign name,Day,amount_diff_pct,impr_diff_pct,clicks_diff_pct
10,Ikonic -Scalp-Massager-Amazon-1-Nov2025,2025-11-11,NaN,NaN,NaN


## Step 10: Final Summary Report

In [11]:
print("="*80)
print(f"ADS PLATFORM DATA VALIDATION SUMMARY (with {THRESHOLD_PERCENT}% threshold)")
print("="*80)
print(f"\nAnalysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Create summary table
summary_data = [
    ['Overall Totals', 5, overall_comparison['Match'].sum(), 5 - overall_comparison['Match'].sum()],
    ['By Date', len(date_comparison), date_comparison['perfect_match'].sum(), 
     (~date_comparison['perfect_match']).sum()],
    ['By Campaign', len(campaign_comparison), campaign_comparison['perfect_match'].sum(), 
     (~campaign_comparison['perfect_match']).sum()],
    ['By Platform', len(platform_comparison), platform_comparison['perfect_match'].sum(), 
     (~platform_comparison['perfect_match']).sum()],
    ['By Placement (Normalized)', len(placement_comparison), placement_comparison['perfect_match'].sum(), 
     (~placement_comparison['perfect_match']).sum()],
    ['By Campaign+Date', len(camp_date_comparison), camp_date_comparison['perfect_match'].sum(), 
     (~camp_date_comparison['perfect_match']).sum()]
]

summary_df = pd.DataFrame(summary_data, 
                         columns=['Segment Type', 'Total Segments', 'Matches', 'Exceeds Threshold'])
summary_df['Match %'] = (summary_df['Matches'] / summary_df['Total Segments'] * 100).round(2)

print("\n")
display(summary_df)

# Overall assessment
total_segments = summary_df['Total Segments'].sum()
total_matches = summary_df['Matches'].sum()
overall_match_pct = (total_matches / total_segments * 100)

print("\n" + "="*80)
print(f"OVERALL MATCH RATE (within {THRESHOLD_PERCENT}%): {total_matches}/{total_segments} ({overall_match_pct:.1f}%)")
print("="*80)

if overall_match_pct == 100:
    print(f"\n✓✓✓ PERFECT VALIDATION! All segments within {THRESHOLD_PERCENT}% threshold! ✓✓✓")
elif overall_match_pct >= 95:
    print(f"\n✓ EXCELLENT! {overall_match_pct:.1f}% of segments within {THRESHOLD_PERCENT}% threshold")
elif overall_match_pct >= 80:
    print(f"\n⚠ GOOD: {overall_match_pct:.1f}% within threshold. Some segments need review.")
else:
    print(f"\n⚠ ATTENTION: Only {overall_match_pct:.1f}% within {THRESHOLD_PERCENT}% threshold. Review required.")

print("\n" + "-"*80)
print("KEY INSIGHTS:")
print("-"*80)
print(f"• Threshold used: {THRESHOLD_PERCENT}%")
print(f"• Segments passing: {total_matches}/{total_segments}")
print(f"• Segments exceeding threshold: {total_segments - total_matches}")
print(f"• Growth rows: {len(growth_df):,}")
print(f"• Gold rows: {len(gold_df):,}")
print(f"• Placement normalization: {len(PLACEMENT_MAPPING)} mappings applied")
print(f"• Platforms validated: {sorted(platform_comparison['Platform'].unique())}")

print("\n" + "="*80)
print("VALIDATION COMPLETE")
print("="*80)

ADS PLATFORM DATA VALIDATION SUMMARY (with 2.0% threshold)

Analysis completed: 2025-12-20 23:10:27




,Segment Type,Total Segments,Matches,Exceeds Threshold,Match %
0,Overall Totals,5,3,2,60.00
1,By Date,30,30,0,100.00
2,By Campaign,4,4,0,100.00
3,By Platform,5,5,0,100.00
4,By Placement (Normalized),20,17,3,85.00
5,By Campaign+Date,101,100,1,99.01



OVERALL MATCH RATE (within 2.0%): 159/165 (96.4%)

✓ EXCELLENT! 96.4% of segments within 2.0% threshold

--------------------------------------------------------------------------------
KEY INSIGHTS:
--------------------------------------------------------------------------------
• Threshold used: 2.0%
• Segments passing: 159/165
• Segments exceeding threshold: 6
• Growth rows: 1,264
• Gold rows: 10,263
• Placement normalization: 20 mappings applied
• Platforms validated: ['audience_network', 'facebook', 'instagram', 'messenger', 'threads']

VALIDATION COMPLETE


In [12]:
# =============================================================================
# COMPREHENSIVE HTML REPORT GENERATION FOR ADS PLATFORM VALIDATION
# Add this as a new cell at the end of your ads_platform_validation_with_dashboard.ipynb
# =============================================================================

from datetime import datetime
import json

print("="*80)
print("GENERATING COMPREHENSIVE HTML REPORT")
print("="*80)

# Calculate summary metrics
overall_match_pct = (total_matches / total_segments * 100) if total_segments > 0 else 0
dates_matched = date_comparison['perfect_match'].sum() if 'date_comparison' in dir() and len(date_comparison) > 0 else 0
dates_total = len(date_comparison) if 'date_comparison' in dir() else 0
campaigns_matched = campaign_comparison['perfect_match'].sum() if 'campaign_comparison' in dir() and len(campaign_comparison) > 0 else 0
campaigns_total = len(campaign_comparison) if 'campaign_comparison' in dir() else 0
platforms_matched = platform_comparison['perfect_match'].sum() if 'platform_comparison' in dir() and len(platform_comparison) > 0 else 0
platforms_total = len(platform_comparison) if 'platform_comparison' in dir() else 0
placements_matched = placement_comparison['perfect_match'].sum() if 'placement_comparison' in dir() and len(placement_comparison) > 0 else 0
placements_total = len(placement_comparison) if 'placement_comparison' in dir() else 0

# Prepare data for charts
overall_metrics_labels = []
overall_metrics_growth = []
overall_metrics_gold = []

if 'overall_comparison' in dir():
    for _, row in overall_comparison.iterrows():
        overall_metrics_labels.append(row['Metric'])
        overall_metrics_growth.append(float(row['Growth']))
        overall_metrics_gold.append(float(row['Gold']))

# Match rate data
match_rate_labels = []
match_rate_data = []
for _, row in summary_df.iterrows():
    match_rate_labels.append(row['Segment Type'])
    match_rate_data.append(float(row['Match %']))

# Platform data
platform_labels = []
platform_cost_growth = []
platform_cost_gold = []
platform_impressions_growth = []
platform_impressions_gold = []

if 'platform_comparison' in dir() and len(platform_comparison) > 0:
    for _, row in platform_comparison.iterrows():
        platform_labels.append(row['Platform'])
        platform_cost_growth.append(float(row['amount_growth']))
        platform_cost_gold.append(float(row['amount_gold']))
        platform_impressions_growth.append(float(row['impressions_growth']))
        platform_impressions_gold.append(float(row['impressions_gold']))

# Campaign data (top 10)
campaign_labels = []
campaign_cost_growth = []
campaign_cost_gold = []
campaign_clicks_growth = []
campaign_clicks_gold = []

if 'campaign_comparison' in dir() and len(campaign_comparison) > 0:
    top_campaigns = campaign_comparison.head(10)
    for _, row in top_campaigns.iterrows():
        campaign_labels.append(row['Campaign name'][:30] + '...' if len(row['Campaign name']) > 30 else row['Campaign name'])
        campaign_cost_growth.append(float(row['amount_growth']))
        campaign_cost_gold.append(float(row['amount_gold']))
        campaign_clicks_growth.append(float(row['clicks_growth']))
        campaign_clicks_gold.append(float(row['clicks_gold']))

# Helper function to create table HTML
def create_table_html(df, title, max_rows=None):
    if df is None or len(df) == 0:
        return f'<h2>{title}</h2><p>No data available</p>'
    
    display_df = df.head(max_rows) if max_rows else df
    html = f'<h2>{title}</h2>\n<table>\n<tr>\n'
    
    # Headers
    for col in display_df.columns:
        html += f'<th>{col}</th>\n'
    html += '</tr>\n'
    
    # Rows
    for _, row in display_df.iterrows():
        html += '<tr>\n'
        for col in display_df.columns:
            value = row[col]
            # Add status styling
            if col == 'Status' or col == 'perfect_match':
                if value == True or 'PASS' in str(value):
                    html += f'<td class="pass">✓ PASS</td>\n'
                else:
                    html += f'<td class="fail">✗ FAIL</td>\n'
            else:
                # Format numbers
                if isinstance(value, (int, float)) and col not in ['Date', 'Day', 'Campaign', 'Platform', 'Placement']:
                    if abs(value) >= 1000:
                        html += f'<td>{value:,.2f}</td>\n'
                    else:
                        html += f'<td>{value:.2f}</td>\n'
                else:
                    html += f'<td>{value}</td>\n'
        html += '</tr>\n'
    
    html += '</table>\n'
    
    if max_rows and len(df) > max_rows:
        html += f'<p style="text-align: center; color: #667eea; font-style: italic;">Showing {max_rows} of {len(df)} total rows</p>\n'
    
    return html

# Generate HTML content
html_content = f'''<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Ads Platform Validation Report</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}

        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
            min-height: 100vh;
        }}

        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 20px 60px rgba(0, 0, 0, 0.3);
            overflow: hidden;
        }}

        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }}

        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);
        }}

        .header p {{
            font-size: 1.2em;
            opacity: 0.9;
        }}

        .metrics-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            padding: 30px;
            background: #f8f9fa;
        }}

        .metric-card {{
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            text-align: center;
            transition: transform 0.3s ease;
        }}

        .metric-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 8px 12px rgba(0, 0, 0, 0.15);
        }}

        .metric-card h3 {{
            color: #667eea;
            font-size: 1em;
            margin-bottom: 10px;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}

        .metric-card .value {{
            font-size: 2.5em;
            font-weight: bold;
            color: #2c3e50;
        }}

        .content {{
            padding: 30px;
        }}

        .section {{
            margin-bottom: 40px;
        }}

        .section h2 {{
            color: #667eea;
            font-size: 1.8em;
            margin-bottom: 20px;
            padding-bottom: 10px;
            border-bottom: 3px solid #667eea;
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }}

        th {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px;
            text-align: left;
            font-weight: 600;
            text-transform: uppercase;
            font-size: 0.9em;
            letter-spacing: 0.5px;
        }}

        td {{
            padding: 12px 15px;
            border-bottom: 1px solid #ecf0f1;
        }}

        tr:hover {{
            background-color: #f8f9fa;
        }}

        .pass {{
            color: #27ae60;
            font-weight: bold;
        }}

        .fail {{
            color: #e74c3c;
            font-weight: bold;
        }}

        .footer {{
            background: #2c3e50;
            color: white;
            text-align: center;
            padding: 20px;
            font-size: 0.9em;
        }}

        .dashboard-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(500px, 1fr));
            gap: 30px;
            padding: 30px;
            background: #f8f9fa;
        }}

        .chart-container {{
            background: white;
            padding: 25px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }}

        .chart-container:hover {{
            transform: translateY(-5px);
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.15);
        }}

        .chart-container h3 {{
            color: #667eea;
            font-size: 1.3em;
            margin-bottom: 20px;
            text-align: center;
            font-weight: 600;
        }}

        .chart-wrapper {{
            position: relative;
            height: 350px;
        }}

        .chart-wrapper.small {{
            height: 300px;
        }}

        .chart-wrapper.large {{
            height: 400px;
        }}
    </style>
</head>

<body>
    <div class="container">
        <div class="header">
            <h1>📊 Ads Platform Validation Report</h1>
            <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p>Threshold: {THRESHOLD_PERCENT}% | Placement Normalization: Enabled</p>
        </div>

        <div class="metrics-grid">
            <div class="metric-card">
                <h3>Overall Match Rate</h3>
                <div class="value">{overall_match_pct:.1f}%</div>
            </div>
            <div class="metric-card">
                <h3>Dates Matched</h3>
                <div class="value">{dates_matched}/{dates_total}</div>
            </div>
            <div class="metric-card">
                <h3>Campaigns Matched</h3>
                <div class="value">{campaigns_matched}/{campaigns_total}</div>
            </div>
            <div class="metric-card">
                <h3>Platforms Matched</h3>
                <div class="value">{platforms_matched}/{platforms_total}</div>
            </div>
            <div class="metric-card">
                <h3>Placements Matched</h3>
                <div class="value">{placements_matched}/{placements_total}</div>
            </div>
            <div class="metric-card">
                <h3>Threshold</h3>
                <div class="value">±{THRESHOLD_PERCENT}%</div>
            </div>
        </div>

        <div class="dashboard-grid">
            <div class="chart-container">
                <h3>📊 Overall Metrics Comparison</h3>
                <div class="chart-wrapper">
                    <canvas id="overallMetricsChart"></canvas>
                </div>
            </div>

            <div class="chart-container">
                <h3>🎯 Match Rate Distribution</h3>
                <div class="chart-wrapper small">
                    <canvas id="matchRateChart"></canvas>
                </div>
            </div>

            <div class="chart-container">
                <h3>💻 Platform Performance - Cost</h3>
                <div class="chart-wrapper">
                    <canvas id="platformCostChart"></canvas>
                </div>
            </div>

            <div class="chart-container">
                <h3>💻 Platform Performance - Impressions</h3>
                <div class="chart-wrapper">
                    <canvas id="platformImpressionsChart"></canvas>
                </div>
            </div>

            <div class="chart-container">
                <h3>🎯 Campaign Performance - Cost (Top 10)</h3>
                <div class="chart-wrapper large">
                    <canvas id="campaignCostChart"></canvas>
                </div>
            </div>

            <div class="chart-container">
                <h3>🎯 Campaign Performance - Clicks (Top 10)</h3>
                <div class="chart-wrapper large">
                    <canvas id="campaignClicksChart"></canvas>
                </div>
            </div>
        </div>

        <div class="content">
            <div class="section">
                {create_table_html(overall_comparison, '📊 Overall Totals Comparison') if 'overall_comparison' in dir() else ''}
            </div>

            <div class="section">
                {create_table_html(platform_comparison, '💻 Validation by Platform (All {0} Platforms)'.format(platforms_total)) if 'platform_comparison' in dir() else ''}
            </div>

            <div class="section">
                {create_table_html(placement_comparison, '📍 Validation by Placement (All {0} Placements)'.format(placements_total)) if 'placement_comparison' in dir() else ''}
            </div>

            <div class="section">
                {create_table_html(campaign_comparison, '🎯 Validation by Campaign (All {0} Campaigns)'.format(campaigns_total)) if 'campaign_comparison' in dir() else ''}
            </div>

            <div class="section">
                {create_table_html(date_comparison, '📅 Validation by Date (Top 20)', max_rows=20) if 'date_comparison' in dir() else ''}
            </div>

            <div class="section">
                {create_table_html(summary_df, '📋 Summary') if 'summary_df' in dir() else ''}
            </div>
        </div>

        <div class="footer">
            <p>Ads Platform Validation Report | Generated with Python & Pandas</p>
            <p>Threshold: ±{THRESHOLD_PERCENT}% | Overall Match Rate: {overall_match_pct:.1f}%</p>
            <p>Placement Normalization: {len(PLACEMENT_MAPPING)} mappings applied</p>
        </div>
    </div>

    <script>
        // Chart.js default configuration
        Chart.defaults.font.family = "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif";
        Chart.defaults.color = '#2c3e50';

        // Color palette
        const colors = {{
            primary: '#667eea',
            secondary: '#764ba2',
            success: '#27ae60',
            danger: '#e74c3c',
            warning: '#f39c12',
            info: '#3498db',
            purple: '#9b59b6',
            orange: '#e67e22',
            teal: '#1abc9c',
            pink: '#e91e63'
        }};

        const gradientColors = [
            'rgba(102, 126, 234, 0.8)',
            'rgba(118, 75, 162, 0.8)',
            'rgba(52, 152, 219, 0.8)',
            'rgba(155, 89, 182, 0.8)',
            'rgba(26, 188, 156, 0.8)',
            'rgba(243, 156, 18, 0.8)',
            'rgba(230, 126, 34, 0.8)',
            'rgba(231, 76, 60, 0.8)',
            'rgba(39, 174, 96, 0.8)',
            'rgba(233, 30, 99, 0.8)'
        ];

        // 1. Overall Metrics Comparison Chart
        const overallMetricsCtx = document.getElementById('overallMetricsChart').getContext('2d');
        new Chart(overallMetricsCtx, {{
            type: 'bar',
            data: {{
                labels: {json.dumps(overall_metrics_labels)},
                datasets: [{{
                    label: 'Growth',
                    data: {json.dumps(overall_metrics_growth)},
                    backgroundColor: colors.primary,
                    borderColor: colors.primary,
                    borderWidth: 2
                }}, {{
                    label: 'Gold',
                    data: {json.dumps(overall_metrics_gold)},
                    backgroundColor: colors.warning,
                    borderColor: colors.warning,
                    borderWidth: 2
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        display: true,
                        position: 'top',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                let label = context.dataset.label || '';
                                if (label) {{
                                    label += ': ';
                                }}
                                label += context.parsed.y.toLocaleString();
                                return label;
                            }}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        ticks: {{
                            callback: function (value) {{
                                return value.toLocaleString();
                            }}
                        }}
                    }}
                }}
            }}
        }});

        // 2. Match Rate Distribution (Doughnut Chart)
        const matchRateCtx = document.getElementById('matchRateChart').getContext('2d');
        new Chart(matchRateCtx, {{
            type: 'doughnut',
            data: {{
                labels: {json.dumps(match_rate_labels)},
                datasets: [{{
                    data: {json.dumps(match_rate_data)},
                    backgroundColor: gradientColors,
                    borderWidth: 3,
                    borderColor: '#fff'
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'bottom',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                return context.label + ': ' + context.parsed + '%';
                            }}
                        }}
                    }}
                }}
            }}
        }});

        // 3. Platform Performance - Cost
        const platformCostCtx = document.getElementById('platformCostChart').getContext('2d');
        new Chart(platformCostCtx, {{
            type: 'bar',
            data: {{
                labels: {json.dumps(platform_labels)},
                datasets: [{{
                    label: 'Growth',
                    data: {json.dumps(platform_cost_growth)},
                    backgroundColor: gradientColors.slice(0, {len(platform_labels)}),
                    borderWidth: 0
                }}, {{
                    label: 'Gold',
                    data: {json.dumps(platform_cost_gold)},
                    backgroundColor: gradientColors.slice(5, {5 + len(platform_labels)}),
                    borderWidth: 0
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        display: true,
                        position: 'top',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                return context.dataset.label + ': ₹' + context.parsed.y.toLocaleString();
                            }}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        ticks: {{
                            callback: function (value) {{
                                return '₹' + value.toLocaleString();
                            }}
                        }}
                    }}
                }}
            }}
        }});

        // 4. Platform Performance - Impressions
        const platformImpressionsCtx = document.getElementById('platformImpressionsChart').getContext('2d');
        new Chart(platformImpressionsCtx, {{
            type: 'line',
            data: {{
                labels: {json.dumps(platform_labels)},
                datasets: [{{
                    label: 'Growth',
                    data: {json.dumps(platform_impressions_growth)},
                    borderColor: colors.primary,
                    backgroundColor: 'rgba(102, 126, 234, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 6,
                    pointHoverRadius: 8
                }}, {{
                    label: 'Gold',
                    data: {json.dumps(platform_impressions_gold)},
                    borderColor: colors.warning,
                    backgroundColor: 'rgba(243, 156, 18, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 6,
                    pointHoverRadius: 8
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        display: true,
                        position: 'top',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                return context.dataset.label + ': ' + context.parsed.y.toLocaleString();
                            }}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        ticks: {{
                            callback: function (value) {{
                                return value.toLocaleString();
                            }}
                        }}
                    }}
                }}
            }}
        }});

        // 5. Campaign Performance - Cost (Top 10)
        const campaignCostCtx = document.getElementById('campaignCostChart').getContext('2d');
        new Chart(campaignCostCtx, {{
            type: 'bar',
            data: {{
                labels: {json.dumps(campaign_labels)},
                datasets: [{{
                    label: 'Growth',
                    data: {json.dumps(campaign_cost_growth)},
                    backgroundColor: colors.primary,
                    borderWidth: 0
                }}, {{
                    label: 'Gold',
                    data: {json.dumps(campaign_cost_gold)},
                    backgroundColor: colors.warning,
                    borderWidth: 0
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                indexAxis: 'y',
                plugins: {{
                    legend: {{
                        display: true,
                        position: 'top',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                return context.dataset.label + ': ₹' + context.parsed.x.toLocaleString();
                            }}
                        }}
                    }}
                }},
                scales: {{
                    x: {{
                        beginAtZero: true,
                        ticks: {{
                            callback: function (value) {{
                                return '₹' + value.toLocaleString();
                            }}
                        }}
                    }}
                }}
            }}
        }});

        // 6. Campaign Performance - Clicks (Top 10)
        const campaignClicksCtx = document.getElementById('campaignClicksChart').getContext('2d');
        new Chart(campaignClicksCtx, {{
            type: 'bar',
            data: {{
                labels: {json.dumps(campaign_labels)},
                datasets: [{{
                    label: 'Growth',
                    data: {json.dumps(campaign_clicks_growth)},
                    backgroundColor: colors.info,
                    borderWidth: 0
                }}, {{
                    label: 'Gold',
                    data: {json.dumps(campaign_clicks_gold)},
                    backgroundColor: colors.success,
                    borderWidth: 0
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                indexAxis: 'y',
                plugins: {{
                    legend: {{
                        display: true,
                        position: 'top',
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function (context) {{
                                return context.dataset.label + ': ' + context.parsed.x.toLocaleString();
                            }}
                        }}
                    }}
                }},
                scales: {{
                    x: {{
                        beginAtZero: true,
                        ticks: {{
                            callback: function (value) {{
                                return value.toLocaleString();
                            }}
                        }}
                    }}
                }}
            }}
        }});
    </script>
</body>

</html>'''

# Save HTML file
with open('ads_platform_validation_report.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("✓ HTML report saved as 'ads_platform_validation_report.html'")

# Open in browser
import webbrowser
import os
html_path = os.path.abspath('ads_platform_validation_report.html')
webbrowser.open('file://' + html_path)

print("\n" + "="*80)
print("REPORT GENERATION COMPLETE")
print("="*80)
print(f"\nFile created: ads_platform_validation_report.html")
print(f"The HTML report has been opened in your default browser.")
print(f"\nReport includes:")
print(f"  • Interactive charts using Chart.js")
print(f"  • Overall metrics comparison")
print(f"  • Match rate distribution")
print(f"  • Platform and Campaign performance charts")
print(f"  • Detailed validation tables")


GENERATING COMPREHENSIVE HTML REPORT
✓ HTML report saved as 'ads_platform_validation_report.html'

REPORT GENERATION COMPLETE

File created: ads_platform_validation_report.html
The HTML report has been opened in your default browser.

Report includes:
  • Interactive charts using Chart.js
  • Overall metrics comparison
  • Match rate distribution
  • Platform and Campaign performance charts
  • Detailed validation tables
